In [ ]:
%pylab inline
import time
import pandas as pd
import pysumma as ps
import xarray as xr
from matplotlib import cm
import itertools
import seaborn as sns
from pathlib import Path
import matplotlib.dates as mdates

from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from pysumma.plotting.utils import justify
sns.set_context('poster')
mpl.style.use('seaborn-bright')
mpl.rcParams['figure.figsize'] = (18, 12)

color_1l  = 'lightseagreen'
colors_2l = ['wheat', 'orange', 'peru']
colors_3l = ['skyblue', 'dodgerblue', 'royalblue']
colors_4l = ['violet', 'deeppink', 'crimson']

dt_list = ['+0.0K', '+2.0K', '+4.0K']
model_list = ['JRDN', 'CLM', '2L_thin', '2L_mid', '2L_thick', '3L_thin', '3L_mid', '3L_thick', '4L_thin', '4L_mid', '4L_thick']
site_list = ['Dana', 'Reynolds_Creek', 'Col-de-Port']

In [ ]:
dana_obs = pd.read_csv('../obs_data/dana_obs.csv', 
                       index_col=0, skipinitialspace=True, parse_dates=True)
cdp_obs = xr.open_dataset('../obs_data/coldeport_obs.nc')
reynolds_obs = xr.open_dataset('../obs_data/reynolds_obs.nc')

dana = xr.open_dataset('../processed/dana_perturbations.nc')
dana['time'] = dana.time.dt.round('H')
coldeport = xr.open_dataset('../processed/coldeport_perturbations.nc')
coldeport['time'] = coldeport.time.dt.round('H')
reynolds = xr.open_dataset('../processed/reynolds_perturbations.nc')
reynolds['time'] = reynolds.time.dt.round('H')

In [ ]:
dana_ts = slice('2009-10-01', '2010-08-01')
dana_swe = dana['scalarSWE'].sel(dt='+0.0K', time=dana_ts).resample(time='D').mean()
reynolds_ts = slice('2003-10-01', '2004-06-01')
reynolds_swe = reynolds['scalarSWE'].sel(dt='+0.0K', time=reynolds_ts).resample(time='D').mean()
coldeport_ts = slice('2000-10-01', '2001-09-01')
coldeport_swe = coldeport['scalarSWE'].sel(dt='+0.0K', time=coldeport_ts).resample(time='D').mean()

dana_mintemp = dana['airtemp'].sel(dt='+0.0K', time=dana_ts).resample(time='D').min() - 273.16
dana_maxtemp = dana['airtemp'].sel(dt='+0.0K', time=dana_ts).resample(time='D').max() - 273.16
dana_depth =   dana['scalarSnowDepth'].sel(dt='+0.0K', time=dana_ts).resample(time='D').mean()
dana_snotemp = dana['scalarSnowTemp'].sel(dt='+0.0K', time=dana_ts).resample(time='D').mean() - 273.16

coldeport_mintemp = coldeport['airtemp'].sel(dt='+0.0K', time=coldeport_ts).resample(time='D').min() - 273.16
coldeport_maxtemp = coldeport['airtemp'].sel(dt='+0.0K', time=coldeport_ts).resample(time='D').max() - 273.16
coldeport_depth =   coldeport['scalarSnowDepth'].sel(dt='+0.0K', time=coldeport_ts).resample(time='D').mean()
coldeport_snotemp = coldeport['scalarSnowTemp'].sel(dt='+0.0K', time=coldeport_ts).resample(time='D').mean() - 273.16

reynolds_mintemp = reynolds['airtemp'].sel(dt='+0.0K', time=reynolds_ts).resample(time='D').min() - 273.16
reynolds_maxtemp = reynolds['airtemp'].sel(dt='+0.0K', time=reynolds_ts).resample(time='D').max() - 273.16
reynolds_depth =   reynolds['scalarSnowDepth'].sel(dt='+0.0K', time=reynolds_ts).resample(time='D').mean()
reynolds_snotemp = reynolds['scalarSnowTemp'].sel(dt='+0.0K', time=reynolds_ts).resample(time='D').mean()- 273.16

In [ ]:
mpl.rcParams['figure.figsize'] = (24, 15)
sizes = {'height_ratios': [3, 3, 3 ,1]}
fig, axes = plt.subplots(4, 3, gridspec_kw=sizes, sharey='row', sharex='col')

#  Plotting for Dana Meadows
axes[3, 0].fill_between(dana_mintemp.time.values, dana_mintemp, dana_maxtemp, alpha=0.5)
axes[3, 0].axhline(0, color='black')
axes[3, 0].set_ylabel('Daily air \n temperature \n range (C)')

dana_snotemp.sel(model='JRDN'    ).plot(color='black',      label='JRDN',     ax=axes[2, 0], linewidth=4);
dana_snotemp.sel(model='CLM'     ).plot(color='lime',       label='CLM',      ax=axes[2, 0], linewidth=4);
dana_snotemp.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[2, 0], linewidth=1.5);
dana_snotemp.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[2, 0], linewidth=1.5);
dana_snotemp.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[2, 0], linewidth=1.5);
dana_snotemp.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[2, 0], linewidth=1.5);
dana_snotemp.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[2, 0], linewidth=1.5);
dana_snotemp.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[2, 0], linewidth=1.5);
dana_snotemp.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[2, 0], linewidth=1.5);
dana_snotemp.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[2, 0], linewidth=1.5);
dana_snotemp.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[2, 0], linewidth=1.5);
axes[2, 0].set_ylabel('Mean snowpack \n temperature (C)')

dana_depth.sel(model='JRDN'    ).plot(color='black',      label='JRDN',     ax=axes[1, 0], linewidth=4);
dana_depth.sel(model='CLM'     ).plot(color='lime',       label='CLM',      ax=axes[1, 0], linewidth=4);
dana_depth.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[1, 0], linewidth=1.5);
dana_depth.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[1, 0], linewidth=1.5);
dana_depth.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[1, 0], linewidth=1.5);
dana_depth.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[1, 0], linewidth=1.5);
dana_depth.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[1, 0], linewidth=1.5);
dana_depth.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[1, 0], linewidth=1.5);
dana_depth.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[1, 0], linewidth=1.5);
dana_depth.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[1, 0], linewidth=1.5);
dana_depth.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[1, 0], linewidth=1.5);
axes[1, 0].set_ylabel('Snow depth (m)')

dana_swe.sel(model='JRDN'    ).plot(color='black',      label='JRDN',     ax=axes[0, 0], linewidth=4);
dana_swe.sel(model='CLM'     ).plot(color='lime',       label='CLM',      ax=axes[0, 0], linewidth=4);
dana_swe.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[0, 0], linewidth=1.5);
dana_swe.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[0, 0], linewidth=1.5);
dana_swe.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[0, 0], linewidth=1.5);
dana_swe.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[0, 0], linewidth=1.5);
dana_swe.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[0, 0], linewidth=1.5);
dana_swe.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[0, 0], linewidth=1.5);
dana_swe.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[0, 0], linewidth=1.5);
dana_swe.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[0, 0], linewidth=1.5);
dana_swe.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[0, 0], linewidth=1.5);
axes[0, 0].set_ylabel('SWE (mm)')
axes[0, 0].set_xlabel('')
axes[0, 0].set_title('Dana Meadows')


#  Plotting for Col-de-Port
axes[3, 1].fill_between(coldeport_mintemp.time.values, coldeport_mintemp, coldeport_maxtemp, alpha=0.5)
axes[3, 1].axhline(0, color='black')

coldeport_snotemp.sel(model='JRDN'    ).plot(color='black',      label='JRDN',     ax=axes[2, 1], linewidth=4);
coldeport_snotemp.sel(model='CLM'     ).plot(color='lime',       label='CLM',      ax=axes[2, 1], linewidth=4);
coldeport_snotemp.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[2, 1], linewidth=1.5);
coldeport_snotemp.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[2, 1], linewidth=1.5);
coldeport_snotemp.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[2, 1], linewidth=1.5);
coldeport_snotemp.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[2, 1], linewidth=1.5);
coldeport_snotemp.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[2, 1], linewidth=1.5);
coldeport_snotemp.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[2, 1], linewidth=1.5);
coldeport_snotemp.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[2, 1], linewidth=1.5);
coldeport_snotemp.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[2, 1], linewidth=1.5);
coldeport_snotemp.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[2, 1], linewidth=1.5);

coldeport_depth.sel(model='JRDN'    ).plot(color='black',      label='JRDN',     ax=axes[1, 1], linewidth=4);
coldeport_depth.sel(model='CLM'     ).plot(color='lime',       label='CLM',      ax=axes[1, 1], linewidth=4);
coldeport_depth.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[1, 1], linewidth=1.5);
coldeport_depth.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[1, 1], linewidth=1.5);
coldeport_depth.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[1, 1], linewidth=1.5);
coldeport_depth.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[1, 1], linewidth=1.5);
coldeport_depth.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[1, 1], linewidth=1.5);
coldeport_depth.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[1, 1], linewidth=1.5);
coldeport_depth.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[1, 1], linewidth=1.5);
coldeport_depth.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[1, 1], linewidth=1.5);
coldeport_depth.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[1, 1], linewidth=1.5);

coldeport_swe.sel(model='JRDN'    ).plot(color='black',      label='JRDN',     ax=axes[0, 1], linewidth=4);
coldeport_swe.sel(model='CLM'     ).plot(color='lime',       label='CLM',      ax=axes[0, 1], linewidth=4);
coldeport_swe.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[0, 1], linewidth=1.5);
coldeport_swe.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[0, 1], linewidth=1.5);
coldeport_swe.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[0, 1], linewidth=1.5);
coldeport_swe.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[0, 1], linewidth=1.5);
coldeport_swe.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[0, 1], linewidth=1.5);
coldeport_swe.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[0, 1], linewidth=1.5);
coldeport_swe.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[0, 1], linewidth=1.5);
coldeport_swe.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[0, 1], linewidth=1.5);
coldeport_swe.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[0, 1], linewidth=1.5);
axes[0, 1].set_ylabel('')
axes[1, 1].set_ylabel('')
axes[0, 1].set_xlabel('')
axes[0, 1].set_title('Col de Porte')

#  Plotting for Reynolds Creek
axes[3, 2].fill_between(reynolds_mintemp.time.values, reynolds_mintemp, reynolds_maxtemp, alpha=0.5)
axes[3, 2].axhline(0, color='black')

reynolds_snotemp.sel(model='JRDN'    ).plot(color='black',      label='SNTHERM-like',     ax=axes[2, 2], linewidth=4);
reynolds_snotemp.sel(model='CLM'     ).plot(color='lime',       label='CLM-like',      ax=axes[2, 2], linewidth=4);
reynolds_snotemp.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[2, 2], linewidth=1.5);
reynolds_snotemp.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[2, 2], linewidth=1.5);
reynolds_snotemp.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[2, 2], linewidth=1.5);
reynolds_snotemp.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[2, 2], linewidth=1.5);
reynolds_snotemp.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[2, 2], linewidth=1.5);
reynolds_snotemp.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[2, 2], linewidth=1.5);
reynolds_snotemp.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[2, 2], linewidth=1.5);
reynolds_snotemp.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[2, 2], linewidth=1.5);
reynolds_snotemp.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[2, 2], linewidth=1.5);

reynolds_depth.sel(model='JRDN'    ).plot(color='black',      label='SNTHERM-like',     ax=axes[1, 2], linewidth=4);
reynolds_depth.sel(model='CLM'     ).plot(color='lime',       label='CLM-like',      ax=axes[1, 2], linewidth=4);
reynolds_depth.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[1, 2], linewidth=1.5);
reynolds_depth.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[1, 2], linewidth=1.5);
reynolds_depth.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[1, 2], linewidth=1.5);
reynolds_depth.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[1, 2], linewidth=1.5);
reynolds_depth.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[1, 2], linewidth=1.5);
reynolds_depth.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[1, 2], linewidth=1.5);
reynolds_depth.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[1, 2], linewidth=1.5);
reynolds_depth.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[1, 2], linewidth=1.5);
reynolds_depth.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[1, 2], linewidth=1.5);

reynolds_swe.sel(model='JRDN'    ).plot(color='black',      label='SNTHERM-like',     ax=axes[0, 2], linewidth=4);
reynolds_swe.sel(model='CLM'     ).plot(color='lime',       label='CLM-like',      ax=axes[0, 2], linewidth=4);
reynolds_swe.sel(model='2L_thin' ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[0, 2], linewidth=1.5);
reynolds_swe.sel(model='2L_mid'  ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[0, 2], linewidth=1.5);
reynolds_swe.sel(model='2L_thick').plot(color=colors_2l[2], label='2l_thick', ax=axes[0, 2], linewidth=1.5);
reynolds_swe.sel(model='3L_thin' ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[0, 2], linewidth=1.5);
reynolds_swe.sel(model='3L_mid'  ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[0, 2], linewidth=1.5);
reynolds_swe.sel(model='3L_thick').plot(color=colors_3l[2], label='3l_thick', ax=axes[0, 2], linewidth=1.5);
reynolds_swe.sel(model='4L_thin' ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[0, 2], linewidth=1.5);
reynolds_swe.sel(model='4L_mid'  ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[0, 2], linewidth=1.5);
reynolds_swe.sel(model='4L_thick').plot(color=colors_4l[2], label='4l_thick', ax=axes[0, 2], linewidth=1.5);
axes[0, 2].set_ylabel('')
axes[1, 2].set_ylabel('')
axes[0, 2].set_xlabel('')
axes[2, 1].set_ylabel('')
axes[2, 2].set_ylabel('')
axes[0, 2].set_title('Reynolds Creek')
axes[1, 0].set_title('')
axes[1, 1].set_title('')
axes[1, 2].set_title('')
axes[2, 0].set_title('')
axes[2, 1].set_title('')
axes[2, 2].set_title('')

plt.tight_layout(pad=-0.1)
fig.autofmt_xdate()
axes[0, 2].legend(bbox_to_anchor=(1., 1.))

In [ ]:
mpl.rcParams['figure.figsize'] = (20, 6)
fig, axes = plt.subplots(1, 3, sharey=True)

dana_ts      = slice('2009-10-28 06', '2010-06-15 18')
coldeport_ts = slice('2000-11-15 06', '2001-06-10 18')
reynolds_ts  = slice('2003-12-14 06', '2004-04-01 18')

dana_snotemp = 1000 * dana['scalarSnowDensity'].sel(dt='+0.0K', time=dana_ts) 
coldeport_snotemp = 1000 * coldeport['scalarSnowDensity'].sel(dt='+0.0K', time=coldeport_ts)
reynolds_snotemp = 1000 * reynolds['scalarSnowDensity'].sel(dt='+0.0K', time=reynolds_ts)

ms = 7
# Plotting for dana meadows
#dana_airtemp.plot(color='black',      label='Air temp',     ax=axes[0], linewidth=6);
(dana_snotemp.sel(model='JRDN'    ) ).plot(color='black',       label='SNTHERM-like',      ax=axes[0], linewidth=4);
(dana_snotemp.sel(model='CLM'     ) ).plot(color='lime',       label='CLM-like',      ax=axes[0], linewidth=4);
(dana_snotemp.sel(model='2L_thin' ) ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[0], linewidth=1.5);
(dana_snotemp.sel(model='2L_mid'  ) ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[0], linewidth=1.5);
(dana_snotemp.sel(model='2L_thick') ).plot(color=colors_2l[2], label='2l_thick', ax=axes[0], linewidth=1.5);
(dana_snotemp.sel(model='3L_thin' ) ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[0], linewidth=1.5);
(dana_snotemp.sel(model='3L_mid'  ) ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[0], linewidth=1.5);
(dana_snotemp.sel(model='3L_thick') ).plot(color=colors_3l[2], label='3l_thick', ax=axes[0], linewidth=1.5);
(dana_snotemp.sel(model='4L_thin' ) ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[0], linewidth=1.5);
(dana_snotemp.sel(model='4L_mid'  ) ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[0], linewidth=1.5);
(dana_snotemp.sel(model='4L_thick') ).plot(color=colors_4l[2], label='4l_thick', ax=axes[0], linewidth=1.5);

#  Plotting for Col-de-Port
#coldeport_airtemp.plot(color='black',      label='Air temp',     ax=axes[1], linewidth=6);
(coldeport_snotemp.sel(model='JRDN'    ) ).plot(color='black',       label='SNTHERM-like',      ax=axes[1], linewidth=4);
(coldeport_snotemp.sel(model='CLM'     ) ).plot(color='lime',       label='CLM-like',      ax=axes[1], linewidth=4);
(coldeport_snotemp.sel(model='2L_thin' ) ).plot(color=colors_2l[0], label='2l_thin',  ax=axes[1], linewidth=1.5);
(coldeport_snotemp.sel(model='2L_mid'  ) ).plot(color=colors_2l[1], label='2l_mid',   ax=axes[1], linewidth=1.5);
(coldeport_snotemp.sel(model='2L_thick') ).plot(color=colors_2l[2], label='2l_thick', ax=axes[1], linewidth=1.5);
(coldeport_snotemp.sel(model='3L_thin' ) ).plot(color=colors_3l[0], label='3l_thin',  ax=axes[1], linewidth=1.5);
(coldeport_snotemp.sel(model='3L_mid'  ) ).plot(color=colors_3l[1], label='3l_mid',   ax=axes[1], linewidth=1.5);
(coldeport_snotemp.sel(model='3L_thick') ).plot(color=colors_3l[2], label='3l_thick', ax=axes[1], linewidth=1.5);
(coldeport_snotemp.sel(model='4L_thin' ) ).plot(color=colors_4l[0], label='4l_thin',  ax=axes[1], linewidth=1.5);
(coldeport_snotemp.sel(model='4L_mid'  ) ).plot(color=colors_4l[1], label='4l_mid',   ax=axes[1], linewidth=1.5);
(coldeport_snotemp.sel(model='4L_thick') ).plot(color=colors_4l[2], label='4l_thick', ax=axes[1], linewidth=1.5);

# Plotting for reynolds creek
(reynolds_snotemp.sel(model='JRDN'    )).plot(color='black',      label='SNTHERM-like',      ax=axes[2], linewidth=4);
(reynolds_snotemp.sel(model='CLM'     )).plot(color='lime',       label='CLM-like',      ax=axes[2], linewidth=4);
(reynolds_snotemp.sel(model='2L_thin' )).plot(color=colors_2l[0], label='2l_thin',  ax=axes[2], linewidth=1.5);
(reynolds_snotemp.sel(model='2L_mid'  )).plot(color=colors_2l[1], label='2l_mid',   ax=axes[2], linewidth=1.5);
(reynolds_snotemp.sel(model='2L_thick')).plot(color=colors_2l[2], label='2l_thick', ax=axes[2], linewidth=1.5);
(reynolds_snotemp.sel(model='3L_thin' )).plot(color=colors_3l[0], label='3l_thin',  ax=axes[2], linewidth=1.5);
(reynolds_snotemp.sel(model='3L_mid'  )).plot(color=colors_3l[1], label='3l_mid',   ax=axes[2], linewidth=1.5);
(reynolds_snotemp.sel(model='3L_thick')).plot(color=colors_3l[2], label='3l_thick', ax=axes[2], linewidth=1.5);
(reynolds_snotemp.sel(model='4L_thin' )).plot(color=colors_4l[0], label='4l_thin',  ax=axes[2], linewidth=1.5);
(reynolds_snotemp.sel(model='4L_mid'  )).plot(color=colors_4l[1], label='4l_mid',   ax=axes[2], linewidth=1.5);
(reynolds_snotemp.sel(model='4L_thick')).plot(color=colors_4l[2], label='4l_thick', ax=axes[2], linewidth=1.5);

axes[0].set_title('Dana Meadows')
axes[1].set_title('Col de Porte')
axes[2].set_title('Reynolds Creek')

axes[0].set_ylabel(r'Snow density ($kg / m^3$)')
axes[1].set_ylabel('')
axes[2].set_ylabel('')

axes[0].set_xlabel('')
axes[1].set_xlabel('')
axes[2].set_xlabel('')
print(xticks()[1])


#plt.tight_layout(pad=-1.1)
axes[2].fmt_xdata = mdates.DateFormatter('%Y-%m')
fig.autofmt_xdate()
axes[2].legend(bbox_to_anchor=(1., 1.))

In [ ]:
mpl.rcParams['figure.figsize'] = (20, 12)
fig, axes = plt.subplots(2, 3, sharey='row')
axes = axes.flatten()

dana_ts      = slice('2010-01-15 06', '2010-01-16 18')
coldeport_ts = slice('2001-01-15 06', '2001-01-16 18')
reynolds_ts  = slice('2004-01-15 06', '2004-01-16 18')

dana_airtemp = dana['airtemp'].sel(time=dana_ts) - 273.16
dana_snotemp = dana['scalarSurfaceTemp'].sel(dt='+0.0K', time=dana_ts) - 273.16

coldeport_airtemp = coldeport['airtemp'].sel(time=coldeport_ts) - 273.16
coldeport_snotemp = coldeport['scalarSurfaceTemp'].sel(dt='+0.0K', time=coldeport_ts) - 273.16

reynolds_airtemp = reynolds['airtemp'].sel(time=reynolds_ts) - 273.16
reynolds_snotemp = reynolds['scalarSurfaceTemp'].sel(dt='+0.0K', time=reynolds_ts) - 273.16

ms = 7
# Plotting for dana meadows

(dana_snotemp.sel(model='JRDN')).plot(color='black',       label='CLM',      ax=axes[0], linewidth=4);

#dana_airtemp.plot(color='black',      label='Air temp',     ax=axes[0], linewidth=6);
(dana_snotemp.sel(model='CLM'     ) - dana_snotemp.sel(model='JRDN')).plot(color='lime',       label='CLM-like',      ax=axes[3], linewidth=4);
(dana_snotemp.sel(model='2L_thin' ) - dana_snotemp.sel(model='JRDN')).plot(color=colors_2l[0], label='2l_thin',  ax=axes[3],marker='o', markersize=ms, linewidth=1.5);
(dana_snotemp.sel(model='2L_mid'  ) - dana_snotemp.sel(model='JRDN')).plot(color=colors_2l[1], label='2l_mid',   ax=axes[3],marker='s', markersize=ms, linewidth=1.5);
(dana_snotemp.sel(model='2L_thick') - dana_snotemp.sel(model='JRDN')).plot(color=colors_2l[2], label='2l_thick', ax=axes[3],marker='d', markersize=ms, linewidth=1.5);
(dana_snotemp.sel(model='3L_thin' ) - dana_snotemp.sel(model='JRDN')).plot(color=colors_3l[0], label='3l_thin',  ax=axes[3],marker='o', markersize=ms, linewidth=1.5);
(dana_snotemp.sel(model='3L_mid'  ) - dana_snotemp.sel(model='JRDN')).plot(color=colors_3l[1], label='3l_mid',   ax=axes[3],marker='s', markersize=ms, linewidth=1.5);
(dana_snotemp.sel(model='3L_thick') - dana_snotemp.sel(model='JRDN')).plot(color=colors_3l[2], label='3l_thick', ax=axes[3],marker='d', markersize=ms, linewidth=1.5);
(dana_snotemp.sel(model='4L_thin' ) - dana_snotemp.sel(model='JRDN')).plot(color=colors_4l[0], label='4l_thin',  ax=axes[3],marker='o', markersize=ms, linewidth=1.5);
(dana_snotemp.sel(model='4L_mid'  ) - dana_snotemp.sel(model='JRDN')).plot(color=colors_4l[1], label='4l_mid',   ax=axes[3],marker='s', markersize=ms, linewidth=1.5);
(dana_snotemp.sel(model='4L_thick') - dana_snotemp.sel(model='JRDN')).plot(color=colors_4l[2], label='4l_thick', ax=axes[3],marker='d', markersize=ms, linewidth=1.5);

#  Plotting for Col-de-Port
(coldeport_snotemp.sel(model='JRDN')).plot(color='black',       label='',      ax=axes[1], linewidth=4);
#coldeport_airtemp.plot(color='black',      label='Air temp',     ax=axes[1], linewidth=6);
(coldeport_snotemp.sel(model='CLM'     ) - coldeport_snotemp.sel(model='JRDN')).plot(color='lime',       label='CLM-like',      ax=axes[4], linewidth=4);
(coldeport_snotemp.sel(model='2L_thin' ) - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_2l[0], label='2l_thin',  ax=axes[4],marker='o', markersize=ms, linewidth=1.5);
(coldeport_snotemp.sel(model='2L_mid'  ) - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_2l[1], label='2l_mid',   ax=axes[4],marker='s', markersize=ms, linewidth=1.5);
(coldeport_snotemp.sel(model='2L_thick') - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_2l[2], label='2l_thick', ax=axes[4],marker='d', markersize=ms, linewidth=1.5);
(coldeport_snotemp.sel(model='3L_thin' ) - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_3l[0], label='3l_thin',  ax=axes[4],marker='o', markersize=ms, linewidth=1.5);
(coldeport_snotemp.sel(model='3L_mid'  ) - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_3l[1], label='3l_mid',   ax=axes[4],marker='s', markersize=ms, linewidth=1.5);
(coldeport_snotemp.sel(model='3L_thick') - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_3l[2], label='3l_thick', ax=axes[4],marker='d', markersize=ms, linewidth=1.5);
(coldeport_snotemp.sel(model='4L_thin' ) - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_4l[0], label='4l_thin',  ax=axes[4],marker='o', markersize=ms, linewidth=1.5);
(coldeport_snotemp.sel(model='4L_mid'  ) - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_4l[1], label='4l_mid',   ax=axes[4],marker='s', markersize=ms, linewidth=1.5);
(coldeport_snotemp.sel(model='4L_thick') - coldeport_snotemp.sel(model='JRDN')).plot(color=colors_4l[2], label='4l_thick', ax=axes[4],marker='d', markersize=ms, linewidth=1.5);

# Plotting for reynolds creek
(reynolds_snotemp.sel(model='JRDN')).plot(color='black',       label='',      ax=axes[2], linewidth=4);
(reynolds_snotemp.sel(model='CLM'     ) - reynolds_snotemp.sel(model='JRDN')).plot(color='lime',       label='CLM-like',      ax=axes[5], linewidth=4);
(reynolds_snotemp.sel(model='2L_thin' ) - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_2l[0], label='2l_thin',  ax=axes[5], marker='o', markersize=ms, linewidth=1.5);
(reynolds_snotemp.sel(model='2L_mid'  ) - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_2l[1], label='2l_mid',   ax=axes[5], marker='s', markersize=ms, linewidth=1.5);
(reynolds_snotemp.sel(model='2L_thick') - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_2l[2], label='2l_thick', ax=axes[5], marker='d', markersize=ms, linewidth=1.5);
(reynolds_snotemp.sel(model='3L_thin' ) - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_3l[0], label='3l_thin',  ax=axes[5], marker='o', markersize=ms, linewidth=1.5);
(reynolds_snotemp.sel(model='3L_mid'  ) - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_3l[1], label='3l_mid',   ax=axes[5], marker='s', markersize=ms, linewidth=1.5);
(reynolds_snotemp.sel(model='3L_thick') - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_3l[2], label='3l_thick', ax=axes[5], marker='d', markersize=ms, linewidth=1.5);
(reynolds_snotemp.sel(model='4L_thin' ) - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_4l[0], label='4l_thin',  ax=axes[5], marker='o', markersize=ms, linewidth=1.5);
(reynolds_snotemp.sel(model='4L_mid'  ) - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_4l[1], label='4l_mid',   ax=axes[5], marker='s', markersize=ms, linewidth=1.5);
(reynolds_snotemp.sel(model='4L_thick') - reynolds_snotemp.sel(model='JRDN')).plot(color=colors_4l[2], label='4l_thick', ax=axes[5], marker='d', markersize=ms, linewidth=1.5);

axes[0].set_title('Dana Meadows')
axes[1].set_title('Col de Porte')
axes[2].set_title('Reynolds Creek')

axes[3].set_title('')
axes[4].set_title('')
axes[5].set_title('')

axes[0].set_ylabel('SNTHERM-like snow \n surface temperature [C]')
axes[1].set_ylabel('')
axes[2].set_ylabel('')

axes[3].set_ylabel('Difference from SNTHERM-like \n snow surface temperature [C]')
axes[4].set_ylabel('')
axes[5].set_ylabel('')

axes[0].set_xlabel('')
axes[1].set_xlabel('')
axes[2].set_xlabel('')
axes[3].set_xlabel('')
axes[4].set_xlabel('')
axes[5].set_xlabel('')

axes[0].axhline(0, color='grey')
axes[1].axhline(0, color='grey')
axes[2].axhline(0, color='grey')
axes[3].axhline(0, color='grey')
axes[4].axhline(0, color='grey')
axes[5].axhline(0, color='grey')


plt.tight_layout(pad=-1.3)
fig.autofmt_xdate()
axes[-1].legend(bbox_to_anchor=(1., 1.6))

In [ ]:
mpl.rcParams['figure.figsize'] = (20, 12)
fig, axes = plt.subplots(2, 3, sharey='row', sharex=False)
colors = ['powderblue', 'azure', '#ffe119', '#f58231']

dana_ts = slice('2009-10-01', '2010-08-01')
dana_swe = dana['scalarSWE'].sel(time=dana_ts).resample(time='D').mean()
dana_swe.sel(dt='-2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='skyblue',       ax=axes[0,0])
dana_swe.sel(dt='+0.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='paleturquoise', ax=axes[0,0])
dana_swe.sel(dt='+2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#ffe119',       ax=axes[0,0])
dana_swe.sel(dt='+4.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#f58231',       ax=axes[0,0])
dana_ts = slice('2016-10-01', '2017-09-01')
dana_swe = dana['scalarSWE'].sel(time=dana_ts).resample(time='D').mean()
dana_swe.sel(dt='-2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='skyblue',       ax=axes[1,0])
dana_swe.sel(dt='+0.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='paleturquoise', ax=axes[1,0])
dana_swe.sel(dt='+2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#ffe119',       ax=axes[1,0])
dana_swe.sel(dt='+4.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#f58231',       ax=axes[1,0])

coldeport_ts = slice('2002-10-01', '2003-09-01')
coldeport_swe = coldeport['scalarSWE'].sel(time=coldeport_ts).resample(time='D').mean()
coldeport_swe.sel(dt='-2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='skyblue',       ax=axes[0,1])
coldeport_swe.sel(dt='+0.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='paleturquoise', ax=axes[0,1])
coldeport_swe.sel(dt='+2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#ffe119',       ax=axes[0,1])
coldeport_swe.sel(dt='+4.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#f58231',       ax=axes[0,1])
coldeport_ts = slice('1998-09-01', '1999-09-01')
coldeport_swe = coldeport['scalarSWE'].sel(time=coldeport_ts).resample(time='D').mean()
coldeport_swe.sel(dt='-2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='skyblue',       ax=axes[1,1])
coldeport_swe.sel(dt='+0.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='paleturquoise', ax=axes[1,1])
coldeport_swe.sel(dt='+2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#ffe119',       ax=axes[1,1])
coldeport_swe.sel(dt='+4.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#f58231',       ax=axes[1,1])

reynolds_ts = slice('2002-10-01', '2003-06-01')
reynolds_swe = reynolds['scalarSWE'].sel(time=reynolds_ts).resample(time='D').mean()
reynolds_swe.sel(dt='-2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='skyblue', ax=axes[0,2])
reynolds_swe.sel(dt='+0.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='paleturquoise', ax=axes[0,2])
reynolds_swe.sel(dt='+2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#ffe119', ax=axes[0,2])
reynolds_swe.sel(dt='+4.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#f58231', ax=axes[0,2])
reynolds_ts = slice('2003-10-01', '2004-06-01')
reynolds_swe = reynolds['scalarSWE'].sel(time=reynolds_ts).resample(time='D').mean()
reynolds_swe.sel(dt='-2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='skyblue',       ax=axes[1,2])
reynolds_swe.sel(dt='+0.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='paleturquoise', ax=axes[1,2])
reynolds_swe.sel(dt='+2.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#ffe119',       ax=axes[1,2])
reynolds_swe.sel(dt='+4.0K').plot.line(x='time', label='-2.0K', add_legend=False, color='#f58231',       ax=axes[1,2])


axes[0, 0].set_xlabel('')
axes[0, 1].set_xlabel('')
axes[0, 2].set_xlabel('')
axes[1, 0].set_xlabel('')
axes[1, 1].set_xlabel('')
axes[1, 2].set_xlabel('')


axes[0, 0].set_ylabel('')
axes[0, 1].set_ylabel('')
axes[0, 2].set_ylabel('')
axes[1, 0].set_ylabel('')
axes[1, 1].set_ylabel('')
axes[1, 2].set_ylabel('')


axes[0, 0].set_title('Dana Meadows (low snow year)', fontsize=22)
axes[0, 1].set_title('Col de Porte (low snow year)', fontsize=22)
axes[0, 2].set_title('Reynolds Creek (low snow year)', fontsize=22)

axes[1, 0].set_title('Dana Meadows (high snow year)', fontsize=22)
axes[1, 1].set_title('Col de Porte (high snow year)', fontsize=22)
axes[1, 2].set_title('Reynolds Creek (high snow year)', fontsize=22)
#axes[0, 2].legend(bbox_to_anchor=(1., 1.))

plt.setp( axes[0, 0].xaxis.get_majorticklabels(), rotation=45 );
plt.setp( axes[0, 1].xaxis.get_majorticklabels(), rotation=45 );
plt.setp( axes[0, 2].xaxis.get_majorticklabels(), rotation=45 );
plt.setp( axes[1, 0].xaxis.get_majorticklabels(), rotation=45 );
plt.setp( axes[1, 1].xaxis.get_majorticklabels(), rotation=45 );
plt.setp( axes[1, 2].xaxis.get_majorticklabels(), rotation=45 );
fig.text(-0.03, 0.5, r'SWE (mm)', va='center', rotation='vertical', fontsize=22)
colors_all = ['skyblue', 'paleturquoise', '#ffe119', '#f58231']
layers = ['-2.0K', '+0.0K', '+2.0K', '+4.0K']
legend_elements = [Line2D([0], [0], color=c, label=m, markersize=15) for m, c in zip(layers, colors_all)]
plt.legend(handles=legend_elements, bbox_to_anchor=(1., 1.))
plt.tight_layout()

In [ ]:
mpl.rcParams['figure.figsize'] = (24, 8)
fig, axes = plt.subplots(1, 3, sharex=False, sharey=False)
axes = axes.flatten()

date1 = '2004-10-01'
date2 = '2007-08-01'
coldeport['scalarSWE'].sel(dt='+0.0K', time=slice(date1, date2), model='JRDN').plot(color='black', label='SNTHERM-like',  linewidth=4, ax=axes[0]);
cdp_obs['SWE_auto'].sel(time=slice(date1, date2)).plot(color='red', label='Observed', ax=axes[0])
plt.xlabel('')
plt.title('')
plt.legend()

date1 = '2009-10-01'
date2 = '2012-08-01'
dana['scalarSWE'].sel(dt='+0.0K', time=slice(date1, date2), model='JRDN').plot(color='black', label='SNTHERM-like',  linewidth=4, ax=axes[1]);
dana_obs['Dana Meadows Pillow SWE [mm]'][date1:date2].plot(color='red',linewidth=3, label='Observed', ax=axes[1])
plt.xlabel('')
plt.title('')
plt.legend()

date1 = '1999-10-01'
date2 = '2002-08-01'
reynolds['scalarSWE'].sel(dt='+0.0K', time=slice(date1, date2), model='JRDN').plot(color='black', label='SNTHERM-like',  linewidth=4, ax=axes[2]);
reynolds_obs['SWE'].sel(time=slice(date1, date2)).plot(color='red',linewidth=3, label='Observed', ax=axes[2])
plt.xlabel('')
plt.ylabel(r'Snow water equivalent $(kg / m^3)$')
plt.title('')

axes[0].set_ylabel(r'Snow Water Equivalent $(kg / m^3)$')
axes[1].set_ylabel(r'')
axes[2].set_ylabel(r'')
axes[0].set_title(r'Col de Porte')
axes[1].set_title(r'Dana Meadows')
axes[2].set_title(r'Reynolds Creek')
axes[0].set_xlabel(r'')
axes[1].set_xlabel(r'')
axes[2].set_xlabel(r'')
plt.legend()